<h1><center> Running Anujit Sarkar’s <h1 style="color:green;"><center> dada2 </center></h1> microbiome-analysis pipeline </center></h1>
<h3><center> author: J. Oberstaller </center></h3>
<h6><center> date: 9/30/2019 </center></h6>


In [ ]:
knitr::opts_chunk$set(echo = TRUE)
source("microbiome.source.scripts.JO.R")

In [1]:
R.Version()


$platform
[1] "x86_64-apple-darwin13.4.0"

$arch
[1] "x86_64"

$os
[1] "darwin13.4.0"

$system
[1] "x86_64, darwin13.4.0"

$status
[1] ""

$major
[1] "3"

$minor
[1] "5.1"

$year
[1] "2018"

$month
[1] "07"

$day
[1] "02"

$`svn rev`
[1] "74947"

$language
[1] "R"

$version.string
[1] "R version 3.5.1 (2018-07-02)"

$nickname
[1] "Feather Spray"

<h1 style="color:green;">Pipeline-overview</h1>

*Goal*: The purpose of this analysis is to obtain an Amplicon Sequence Variant (ASV) table for all of our microbiome-sample example-data.

*Input data*: We will start with demultiplexed fastq files for all samples. *This analysis is for paired-end data.* Thus, for each sample, there will be two files, named according to Illumina platform conventions:

  1. Forward-reads, named *_R1_001.fastq
  2. Reverse-reads, named *_R2_001.fastq

## Before we begin ##

Before we begin, let's set up a universal directory-structure to keep our analyses organized by project. We'll use the "here" package to make a "microbiome_workshop" folder in our top-level project-directory as our working directory.

    **the "here" package determines your top-level directory-path, whatever it may be, and allows our scripts to be portable without breaking due to file-path errors.**
    
We'll create the following subdirectories in our microbiome_workshop directory at the same time:

  - **Rdata**: this folder will contain our input .fastq.gz files.
  - **Ranalysis**: this folder will contain any R-scripts we create to analyze these data.
  - **Routput**: we will direct any output-files from our analyses to this folder.
  - **Rfigs**: we will direct any figures we generate from our analyses to this folder.

In [ ]:
# create project-directory and subdirectories
init.R.proj.JO(project.name = "microbiome_workshop")

# set the project-directory as the working-directory
setwd(here("microbiome_workshop"))

#### Moving your input-data to your microbiome_workshop/Rdata folder using Finder ####

Next, use the lower-right navigation-pane in RStudio and select (check the box) the project-directory you just created. Then go to the "More" dropdown window and select "Show Folder in New Window". Your folder will open in Finder.

  *Using Finder, open the "Rdata" folder. Move the data you downloaded to the Rdata folder. If the fastq files are in zipped format, they should be* **unzipped** *first by double-clicking the ".zip" file in Finder.*
  
  Now we're ready to start our analyses.
  
  *NOTE: add screenshots to this section*

<h1 style="color:green;">Begin data-analyses with dada2</h1>

Now back to RStudio.

First we'll load the appropriate R package for the analysis (dada2). Dada2 will have automatically been installed when you ran this notebook.

In [ ]:
library(dada2)

<h2 style="color:#0BDCD6;background-color:black;vertical-align:middle;"><center>*Important:* make sure to note the package-version of dada2 you're using! Documenting all software-versions is critical for producing quality, reproducible computational analyses.<center></h2>

In [ ]:
packageVersion("dada2")

Now we're ready to work with our data. First we'll give the path to the directory where all the fastq files are stored a name ("demo_microbiome_fasqfiles"), then we'll check that that fastq source-directory contains all our fastq files.

In [ ]:
# name input-folder path
demo_microbiome_fasqfiles <- "./Rdata"

# Check that the fastq source directory contains all our fastq files
list.files(demo_microbiome_fasqfiles)

Now make two variables to separate and store the forward and the reverse reads:

In [ ]:
demo_F <- sort(list.files(demo_microbiome_fasqfiles, pattern="_R1_001.fastq", full.names = TRUE))
demo_R <- sort(list.files(demo_microbiome_fasqfiles, pattern="_R2_001.fastq", full.names = TRUE))

# Extract filenames of all samples for future steps in the analysis
demo_samplenames <- sapply(strsplit(basename(demo_F), "_"), '[', 1)

## Evaluating data-quality ##

Let's check our data-quality by making plots and viewing them directly in RStudio. Your plots will appear in the RStudio "Plots" pane to the lower-right.

In [ ]:
# First we'll check the forward-reads:
plotQualityProfile(demo_F[1:4], n=1e+06)

# Now let's plot to check the data-quality of our reverse-reads.
plotQualityProfile(demo_R[1:4], n=1e+06)


Let's also output the plots as .pdf files so we can view them later. They'll be saved in the Rfigs directory.
  *Helpful tip: It is important to save any data or figures you generate in R that you want to keep to file; they are not saved when you quit RStudio, and you'll have to regenerate them!*

In [ ]:
## save data-quality plot for forward-reads:
pdf('Rfigs/demo_F_quality.pdf', width = 12, height = 8, pointsize = 8)
plotQualityProfile(demo_F[1:4], n=1e+06)
dev.off()

# save the data-quality plot for our reverse-reads:
pdf('Rfigs/demo_R_quality.pdf', width = 12, height = 8, pointsize = 8)
plotQualityProfile(demo_R[1:4], n=1e+06)
dev.off()

## Filter reads based on data-quality ##

The next step is to filter the sequences appropriately, the parameters for which will depend on the data. 

Conceptually, we will discard the bad reads, trim the ends of the good reads, and then save the trimmed good reads to a new directory.

First we will specify the path and name the output-files to which the good sequences will be written. 
  **the directory and output-files we specify here will be created in the next step (filterAndTrim).**

In [ ]:
# The first step here is to specify the path and name the output-files to which the good sequences will be written. 
  # the directory and output-files we specify here will be created in the next step (filterAndTrim).
demo_goodF <- file.path("Routput/demo_good_filtered", paste0(demo_samplenames, "F_good.fastq.gz"))
demo_goodR <- file.path("Routput/demo_good_filtered", paste0(demo_samplenames, "R_good.fastq.gz"))
names(demo_goodF) <- demo_samplenames
names(demo_goodR) <- demo_samplenames

Now we perform the very important step of filtering and trimming each fastq. 

*These parameters are flexible and should depend on your data!*

In [ ]:

demo_good_proper <- filterAndTrim(demo_F, demo_goodF, demo_R, demo_goodR, trimLeft = c(17, 21), truncLen = c(145, 135), maxN = 0, truncQ = 2, minQ=1, maxEE = c(2, 4), rm.phix = TRUE, n = 1e+5, compress = TRUE, verbose = TRUE)

# save the output of previous step (a summary table indicating how many reads there were for each sample before and after quality-filtering):
write.table(demo_good_proper, "Routput/demo_filteredout.txt", sep = "\t", quote = FALSE)


## Calculate and plot error-rates ##

* **error-rate of what? ** *

Now let's calculate the error-rates for the forward and reverse sequences, plot them directly in RStudio and save to .pdf.

In [ ]:
## forward-reads:
demo_error_F <- learnErrors(demo_goodF, nbases = 1e+07, randomize = TRUE, MAX_CONSIST = 12, multithread = TRUE, verbose = TRUE)
plotErrors(demo_error_F, obs = TRUE, nominalQ = TRUE)

## and reverse-reads:
demo_error_R <- learnErrors(demo_goodR, nbases = 1e+07, randomize = TRUE, MAX_CONSIST = 12, multithread = TRUE, verbose = TRUE)
plotErrors(demo_error_R, obs = TRUE, nominalQ = TRUE)


We'll also save both plots to .pdf in our Rfigs directory for our records:

In [ ]:

pdf('Rfigs/demo_error_F_plot.pdf', width = 10, height = 10, pointsize = 8)
plotErrors(demo_error_F, obs = TRUE, nominalQ = TRUE)
dev.off()

pdf('Rfigs/demo_error_R_plot.pdf', width = 10, height = 10, pointsize = 8)
plotErrors(demo_error_R, obs = TRUE, nominalQ = TRUE)
dev.off()

## Dereplicate sequences in each sample ##

*Notes about why this is necessary*

In [ ]:
# The next step is to dereplicate the sequences in each sample
derep_demo_F <- derepFastq(demo_goodF, n = 1e+06, verbose = TRUE)
derep_demo_R <- derepFastq(demo_goodR, n = 1e+06, verbose = TRUE)

names(derep_demo_F) <- demo_samplenames
names(derep_demo_R) <- demo_samplenames


## Running dada2: Calculating ASVs ##

Now it is time to run the actual dada2 algorithm to determine the ASVs in the dataset.

  ** This step is run separately for forward and reverse sets of paired-end reads **

In [ ]:

demo_dada_F <- dada(derep_demo_F, err=demo_error_F, pool = TRUE, multithread = TRUE)
demo_dada_R <- dada(derep_demo_R, err=demo_error_R, pool = TRUE, multithread = TRUE)

# Let's see how many sequence variants we have got in the forward set
demo_dada_F[[1]]

Next we will merge the forward and reverse sets (the paired-end reads for all our samples), and output a sequence-table of all ASVs.

In [ ]:
demo_merged <- mergePairs(demo_dada_F, derep_demo_F, demo_dada_R, derep_demo_R, minOverlap = 20, maxMismatch = 0, verbose = TRUE)

# Let's make a sequence table of all the ASVs
demo_sequence_table <- makeSequenceTable(demo_merged, orderBy = "abundance")

# We can check the distribution of the ASVs by length
table(nchar(getSequences(demo_sequence_table)))


## Filtering chimeric sequences ##

**Notes about why this is important**

In [ ]:

# Remove chimeric sequences
demo_nochim <- removeBimeraDenovo(demo_sequence_table, method = "consensus", minFoldParentOverAbundance = 1, verbose = TRUE, multithread = TRUE)

# Let's see how many ASVs remain after filtering chimeric sequences:
dim(demo_nochim)

# Let's see the proportion of sequences we retained after filtering for chimeric sequences:
sum(demo_nochim)/sum(demo_sequence_table)


Now we have completed all the filtering, trimming, cleanup etc. to arrive at our final data-set. Here we should check and record how many sequences we retained after each step.
  
  ** This test is important for trouble-shooting purposes. **

In [ ]:
# Create a function to calculate reads retained
fetch_numbers <- function(a) sum(getUniques(a))

# Then apply this function to the output of each step in our pipeline to generate a counts-table of reads remaining after each step
demo_track_steps <- cbind(demo_good_proper, sapply(demo_dada_F, fetch_numbers), sapply(demo_dada_R, fetch_numbers), sapply(demo_merged, fetch_numbers), rowSums(demo_nochim))
colnames(demo_track_steps) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "nochim")
rownames(demo_track_steps) <- demo_samplenames

# And save the output to a new file for our records:
write.table(demo_track_steps, "Routput/demo_filtering_steps_track.txt", sep = "\t", quote = FALSE)


## Assign taxonomy to all ASVs ##

We will next determine taxa present in our samples using the Silva database v.132.

  *You downloaded this file to your Rdata directory previously (Rdata/Silva_db/silva_nr_v132_train_set.fa)*


In [ ]:
demo_taxonomy <- assignTaxonomy(demo_nochim, "Rdata/Silva_db/silva_nr_v132_train_set.fa", minBoot = 80, verbose = TRUE, multithread = TRUE)
write.table(demo_taxonomy, "Routput/demo_taxaout.txt", sep = "\t", quote = FALSE)

Now we will generate output-files critical for further analyses and data-visualization. These include:

    a table summarizing ASVs by taxa
    a fasta-file of all ASVs
    a table of ASV-counts per sample (the OTU-table)


In [ ]:
# Let's create a table by replacing the ASV sequences with ids (ASV_1, ASV_2 etc.) and their corresponding classifications
demo_taxa_summary <- demo_taxonomy
row.names(demo_taxa_summary) <- NULL
head(demo_taxa_summary)

# Let's make a file listing all the ASVs and their sequences in fasta format
demo_asv_seqs <- colnames(demo_nochim)
demo_asv_headers <- vector(dim(demo_nochim)[2], mode = "character")
for (i in 1:dim(demo_nochim)[2]) {demo_asv_headers[i] <- paste(">ASV", i, sep = "_")}
demo_asv.fasta <- c(rbind(demo_asv_headers, demo_asv_seqs))
write(demo_asv.fasta, "Routput/demo_out_asv.fasta")

# At this step, we need to make a table of ASV counts for each sample (which is going to be most important for all statistical analyses)
demo_asv_tab <- t(demo_nochim)
row.names(demo_asv_tab) <- sub(">", "", demo_asv_headers)
write.table(demo_asv_tab, "Routput/demo_asv_counts.tsv", sep = "\t", quote=F, col.names = NA)

# Finally, let's make a table with the taxonomy of all the ASVs
demo_asv_taxa <- demo_taxonomy
row.names(demo_asv_taxa) <- sub(">", "", demo_asv_headers)
write.table(demo_asv_taxa, "Routput/demo_asvs_taxonomy.tsv", sep = "\t", quote=F, col.names = NA)
dim(demo_asv_taxa)


## ** THANK YOU EVERYONE ** ##

## Data visualization : could we add a walkthrough of generating some common microbiome-analysis figures from these data?
